In [36]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS

# https://stackoverflow.com/questions/57451719/since-spark-2-3-the-queries-from-raw-json-csv-files-are-disallowed-when-the-ref
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

FILE_NAME = "tweets.json"

original_data = spark.read.option("multiline","true").json(FILE_NAME).cache()

In [48]:
# get user_id 
all_user_ids = original_data.select("user_id").distinct().rdd.map(lambda row:row[0]).collect()

# create dictionary for user_id
user_id_to_index = dict()
for user_id in all_user_ids:
    user_id_to_index[user_id] = len(user_id_to_index)

def extract_id(rows):
    return [(mention_user[0]) for mention_user in rows[0]]

# extract mentioned user id
all_mention_users = original_data.select("user_mentions") \
                                 .filter("user_mentions is not null") \
                                 .rdd.flatMap(extract_id) \
                                 .distinct().collect()


# create dictionary for mentioned user id
mention_id_to_index = dict()
for mention_user_id in all_mention_users:
    mention_id_to_index[mention_user_id] = len(mention_id_to_index)

sc.broadcast(user_id_to_index)
sc.broadcast(mention_id_to_index)

In [56]:
def extract_data(rows):
    user_id = rows[0]
    return [(user_id, mention_id_to_index[mention_user[0]], 1) for mention_user in rows[1]]

def create_key(rows):
    user_id = rows[0] 
    mention_user = rows[1] 
    counter = rows[2]
    return ((user_id,mention_user), counter)

mention_user = originalData.select("user_id","user_mentions").where("user_mentions is not null")
# extract data
clear_data = mention_user.rdd.map(lambda row:(user_id_to_index[row[0]],row[1])).flatMap(extract_data)
# combine user_id mention_user_id as a key
key_data = clear_data.map(create_key)
# calculate                    
user_mention_data = key_data.reduceByKey(lambda counter1, counter2 : counter1 + counter2) \
                            .map(lambda row : (row[0][0], row[0][1], row[1])) \
                            .toDF(["user_id", "mention_user_id", "counter"])

# https://spark.apache.org/docs/latest/ml-collaborative-filtering.html
# train data
als = ALS(userCol="user_id", itemCol="mention_user_id", ratingCol="counter",coldStartStrategy="drop")
model = als.fit(user_mention_data)       
model_recommend = model.recommendForAllUsers(5).collect()

# print result
for rows in model_recommend:
    print(all_user_ids[rows[0]], ":", end=" ")
    for data in rows[1]:
        print(all_mention_users[data[0]], end="   ")
    print()

123487165 : 24259259   346102488   1244013067790364673   3094649957   21802625   
1380638553353248769 : 24259259   346102488   1244013067790364673   3094649957   21802625   
1216930231082258433 : 1353769946556325889   4018888580   17154865   25562183   33057171   
941035756759502849 : 24259259   26574283   185025785   1004633989   2791419078   
277468009 : 24259259   22429979   9814812   299273962   346102488   
406447628 : 24259259   3094649957   346102488   1244013067790364673   133081348   
22352061 : 528820859   421223083   22352061   94482117   1255482927795834883   
393701691 : 24259259   26574283   185025785   1004633989   2791419078   
2890132003 : 3240396234   17839398   1278694316752490500   42953038   621064274   
1121130373881184256 : 24259259   346102488   1244013067790364673   3094649957   807095   
134266069 : 24259259   26574283   185025785   1004633989   2791419078   
783489062342631424 : 2292688374   3098228941   165275535   99447959   2520717924   
137216640697266177

2764282224 : 24259259   26574283   185025785   1004633989   2791419078   
906697765622878208 : 24259259   346102488   1244013067790364673   3094649957   807095   
497756432 : 5392522   1278694316752490500   881974187589603328   9814812   1643123766   
2191984092 : 24259259   346102488   1244013067790364673   3094649957   807095   
1268667976116514817 : 9814812   842678603305377793   13393052   24259259   149075155   
864258388414398464 : 24259259   380648579   100248020   32828872   3094649957   
1342883774845964288 : 24259259   346102488   1244013067790364673   3094649957   807095   
22934300 : 9814812   842678603305377793   13393052   24259259   149075155   
780639889314369536 : 24259259   346102488   1244013067790364673   3094649957   21802625   
1001470369 : 24259259   380648579   100248020   32828872   3094649957   
1262282601638907905 : 24259259   346102488   1244013067790364673   3094649957   807095   
474571974 : 24259259   3094649957   346102488   1244013067790364673   1330813

2751947681 : 24259259   380648579   100248020   32828872   3094649957   
1350427654659346433 : 24259259   15115280   224329419   866065134   1003107003693137921   
289034668 : 24259259   26574283   185025785   1004633989   2791419078   
709703767 : 24259259   355989081   1236039824282251264   1595526518   939091   
1292967632 : 24259259   56488059   20457806   55060090   250310762   
2848493747 : 947605221416538112   362105466   1130959251671834624   449027230   274233761   
2615533080 : 24259259   346102488   1244013067790364673   3094649957   807095   
236149457 : 24259259   56488059   20457806   231510077   2838696137   
1371757724917100544 : 24259259   355989081   1236039824282251264   1595526518   939091   
56747956 : 37973014   1002260754022748162   33584794   3240396234   2998718043   
1078780581536718848 : 9814812   842678603305377793   13393052   24259259   149075155   
148163990 : 24259259   26574283   185025785   1004633989   2791419078   
1142429395182936064 : 24259259   34

1177070435286097920 : 24259259   26574283   185025785   1004633989   2791419078   
98937081 : 24259259   346102488   1244013067790364673   3094649957   807095   
1336506580699738113 : 9814812   299273962   24259259   22429979   346102488   
56722736 : 24259259   346102488   1244013067790364673   3094649957   807095   
49834525 : 24259259   380648579   100248020   32828872   3094649957   
247448919 : 24259259   26574283   185025785   1004633989   2791419078   
1275719886443077633 : 9814812   114968487   157981564   65201417   80697114   
1245073576715153409 : 24259259   346102488   1244013067790364673   3094649957   807095   
1252611593906515968 : 24259259   346102488   1244013067790364673   3094649957   807095   
1148951598594428928 : 24259259   3094649957   346102488   1244013067790364673   1656958189   
36326893 : 1146329871418843136   36326893   1003895325025865730   42183750   3384987100   
4897701172 : 24259259   26574283   185025785   1004633989   2791419078   
16868756 : 1416705

77692767 : 24259259   200737738   96900937   2976653309   4970411   
1197862862510678016 : 24259259   26574283   185025785   1004633989   2791419078   
1031214881889021952 : 24259259   26574283   185025785   1004633989   2791419078   
1197995707254345729 : 24259259   346102488   1244013067790364673   20457806   56488059   
23921946 : 24259259   380648579   100248020   32828872   3094649957   
21998010 : 24259259   346102488   3094649957   1244013067790364673   21802625   
470484023 : 842678603305377793   121181001   9814812   29959618   431762931   
1388879909712498688 : 24259259   346102488   1244013067790364673   3094649957   807095   
2837351588 : 24259259   346102488   1244013067790364673   3094649957   807095   
839700014909775872 : 24259259   26574283   185025785   1004633989   2791419078   
1057814820 : 24259259   26574283   185025785   1004633989   2791419078   
1041371804944789504 : 24259259   26574283   185025785   1004633989   2791419078   
34811078 : 24259259   346102488   

...

1350145359524233218 : 14167059   17849506   885885503349936128   19380829   866065134   
1115072743 : 24259259   26574283   185025785   1004633989   2791419078   
3218937110 : 5392522   1278694316752490500   881974187589603328   9814812   1643123766   
16516557 : 24259259   346102488   1244013067790364673   3094649957   807095   
1068840824048414721 : 24259259   26574283   185025785   1004633989   2791419078   
2653909938 : 3290364847   132630502   32840323   837335683215798272   24259259   
801545853538369536 : 1309091227   15859912   702013982618558464   1943139204   94482117   
244707638 : 24259259   26574283   185025785   1004633989   2791419078   
1057153154 : 24259259   346102488   1244013067790364673   3094649957   807095   
1309507700926029824 : 24259259   346102488   1244013067790364673   3094649957   807095   
616002513 : 24259259   26574283   185025785   1004633989   2791419078   
2587500907 : 24259259   346102488   1244013067790364673   3094649957   807095   
891195404 : 24

143685204 : 24259259   346102488   1244013067790364673   3094649957   807095   
56897957 : 24259259   3094649957   346102488   1244013067790364673   133081348   
2421228369 : 155062488   20578408   242349332   1037346081619804163   1849172972   
3001455391 : 24259259   380648579   100248020   32828872   3094649957   
16708028 : 24259259   346102488   3094649957   1244013067790364673   21802625   
54763819 : 24259259   346102488   1244013067790364673   3094649957   807095   
409510396 : 24259259   346102488   1244013067790364673   3094649957   807095   
1054040460459569152 : 24259259   355989081   1236039824282251264   1595526518   939091   
40578058 : 24259259   346102488   1244013067790364673   3094649957   807095   
3098052395 : 9814812   24259259   18831926   335060282   299273962   
458526016 : 24259259   3094649957   346102488   1244013067790364673   133081348   
1375218097964130305 : 9814812   842678603305377793   13393052   24259259   149075155   
117455135 : 24259259   38064857

824991126 : 24259259   346102488   3094649957   1244013067790364673   21802625   
560905460 : 24259259   355989081   1236039824282251264   1595526518   939091   
10967182 : 842678603305377793   121181001   9814812   29959618   431762931   
1385285385966612483 : 24259259   346102488   1244013067790364673   3094649957   807095   
2468243359 : 17674244   25146372   318518392   1876968744   15859912   
1246420521148796928 : 24259259   3094649957   346102488   1244013067790364673   133081348   
868106252789751808 : 24259259   380648579   100248020   32828872   3094649957   
112502883 : 24259259   26574283   185025785   1004633989   2791419078   
3184446464 : 24259259   346102488   1244013067790364673   3094649957   807095   
595328000 : 24259259   26574283   185025785   1004633989   2791419078   
363394373 : 24259259   20457806   56488059   1078401427347857408   155585373   
788889923050885120 : 24259259   346102488   1244013067790364673   3094649957   807095   
2743926708 : 320432933   164

992082427 : 5392522   1278694316752490500   881974187589603328   9814812   1643123766   
2601175979 : 1917731   369534928   24259259   346102488   1244013067790364673   
1002857575623188480 : 24259259   26574283   185025785   1004633989   2791419078   
1299401392457539586 : 24259259   26574283   185025785   1004633989   2791419078   
743580907865513985 : 24259259   346102488   1244013067790364673   3094649957   807095   
25255543 : 24259259   346102488   1244013067790364673   3094649957   807095   
47106512 : 9814812   299273962   24259259   22429979   346102488   
295941632 : 24259259   26574283   185025785   1004633989   2791419078   
1121737707938562048 : 24259259   346102488   3094649957   1244013067790364673   21802625   
738567336853020673 : 1917731   369534928   24259259   346102488   1244013067790364673   
856179222 : 24259259   346102488   1244013067790364673   3094649957   807095   
484704769 : 198118653   380185975   274233761   9300262   449027230   
3015988118 : 24259259  

1277723780702277634 : 227429355   24259259   9814812   1206227149   815733290955112448   
1329246230069841921 : 24259259   346102488   1244013067790364673   3094649957   807095   
624827127 : 24259259   346102488   1244013067790364673   3094649957   807095   
1176300448527110146 : 24259259   26574283   185025785   1004633989   2791419078   
822476494538702849 : 24259259   346102488   1244013067790364673   3094649957   807095   
766032497654239232 : 24259259   346102488   1244013067790364673   3094649957   807095   
726838049468985345 : 24259259   346102488   1244013067790364673   3094649957   807095   
929503909 : 24259259   346102488   1244013067790364673   3094649957   807095   
1233107656539283460 : 33584794   3240396234   37973014   1002260754022748162   1688796138   
963574904389099520 : 24259259   346102488   1244013067790364673   3094649957   807095   
343415313 : 24259259   346102488   1244013067790364673   3094649957   807095   
1332269284710637568 : 24259259   346102488   124